In [1]:
import pandas as pd
import numpy as np

import matplotlib
import seaborn as sns

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import pearsonr, spearmanr
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Split last week starting 4-22 two days val two days test

In [2]:
df = pd.read_csv('clean_sales.csv')

In [3]:
df

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,is_instore,product_id,quantity,line_item_amount,unit_price,is_promo
0,1,2019-04-01,12:04:43,3,12.0,558,0,52,1,2.50,2.50,0
1,2,2019-04-01,15:54:39,3,17.0,781,0,27,2,7.00,3.50,0
2,3,2019-04-01,14:34:59,3,17.0,788,1,46,2,5.00,2.50,0
3,4,2019-04-01,16:06:04,3,12.0,683,0,23,2,5.00,2.50,0
4,5,2019-04-01,19:18:37,3,17.0,99,1,34,1,2.45,2.45,0
...,...,...,...,...,...,...,...,...,...,...,...,...
49889,39021,2019-04-29,16:51:58,8,42.0,0,0,30,1,3.00,3.00,0
49890,39022,2019-04-29,16:51:14,8,42.0,8412,1,25,2,4.40,2.20,0
49891,39023,2019-04-29,11:17:36,8,15.0,0,1,31,1,2.20,2.20,0
49892,39024,2019-04-29,15:45:52,8,45.0,8030,0,44,2,5.00,2.50,0


In [6]:
df.transaction_date = pd.to_datetime(df.transaction_date)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49894 entries, 0 to 49893
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_id    49894 non-null  int64         
 1   transaction_date  49894 non-null  datetime64[ns]
 2   transaction_time  49894 non-null  object        
 3   sales_outlet_id   49894 non-null  int64         
 4   staff_id          49894 non-null  float64       
 5   customer_id       49894 non-null  int64         
 6   is_instore        49894 non-null  object        
 7   product_id        49894 non-null  int64         
 8   quantity          49894 non-null  int64         
 9   line_item_amount  49894 non-null  float64       
 10  unit_price        49894 non-null  float64       
 11  is_promo          49894 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(6), object(2)
memory usage: 4.6+ MB


In [9]:
train = df[df.transaction_date < '2019-04-22']

In [10]:
test = df[df.transaction_date >= '2019-04-22']

In [11]:
test

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,is_instore,product_id,quantity,line_item_amount,unit_price,is_promo
36472,27874,2019-04-22,09:42:54,3,20.0,0,1,40,2,7.5,3.75,0
36473,27875,2019-04-22,17:46:43,3,16.0,0,0,28,1,2.0,2.00,0
36474,27876,2019-04-22,16:53:25,3,14.0,0,1,24,1,3.0,3.00,0
36475,27877,2019-04-22,08:27:27,3,14.0,0,1,48,1,2.5,2.50,0
36476,27878,2019-04-22,08:23:33,3,16.0,0,1,51,1,3.0,3.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...
49889,39021,2019-04-29,16:51:58,8,42.0,0,0,30,1,3.0,3.00,0
49890,39022,2019-04-29,16:51:14,8,42.0,8412,1,25,2,4.4,2.20,0
49891,39023,2019-04-29,11:17:36,8,15.0,0,1,31,1,2.2,2.20,0
49892,39024,2019-04-29,15:45:52,8,45.0,8030,0,44,2,5.0,2.50,0
